# Notebook 2: Generate multi-strain models

This notebook follows Notebook 1 in the tutorial, and continues to work on generating multi-strain E.coli models. This notebook utilizes the output of notebook 1 (presence/absence matrix and geneID matrix) to generate draft strain-specific models from the 
reference model. There are two major steps involved: 

1. Deletion of missing genes/reaction from reference model to generate draft models
2. Update gene-protein-reaction rule in each model

In [1]:
#import package needed
import cobra
import pandas as pd
from cobra.io import load_json_model
from glob import glob
from cobra.manipulation.delete import delete_model_genes, remove_genes

## 1. Deletion of missing genes/reaction from reference model 


In [2]:
#Load the base E. coli reconstruction iML1515
model = load_json_model('iML1515.json')
model

<Model iML1515 at 0x7f4ae6ef1828>

In [3]:
## Load the previously generated homology matrix for E. coli strains of interest
hom_matrix=pd.read_csv('ortho_matrix.csv')
hom_matrix=hom_matrix.set_index('Unnamed: 0')

In [4]:
hom_matrix

CP000946.1  CU651637.1  CP002167.1  CU928163.2  CU928164.2
Unnamed: 0                                                            
b2551              1.0         1.0         1.0         1.0         1.0
b0870              1.0         1.0         1.0         1.0         1.0
b3368              1.0         1.0         1.0         1.0         1.0
b2436              1.0         1.0         1.0         1.0         1.0
b3500              1.0         1.0         1.0         1.0         1.0
b0945              1.0         1.0         1.0         1.0         1.0
b4467              1.0         1.0         1.0         1.0         1.0
b4468              1.0         1.0         1.0         1.0         1.0
b2979              1.0         1.0         1.0         1.0         1.0
b3916              1.0         1.0         1.0         1.0         1.0
b1260              1.0         1.0         1.0         1.0         1.0
b1261              1.0         1.0         1.0         1.0         1.0
b3426              1.0         1.0         1.0         1.0         1.0
b2242              1.0         1.0         1.0         1.0         1.0
b2243              1.0         1.0         1.0         1.0         1.0
b2241              1.0         1.0         1.0         1.0         1.0
b4054              1.0         1.0         1.0         1.0         1.0
b3770              1.0         1.0         1.0         1.0         1.0
b3281              1.0         1.0         1.0         1.0         1.0
b1692              1.0         1.0         1.0         1.0         1.0
b0243              1.0         1.0         1.0         1.0         1.0
b1613              1.0         1.0         1.0         1.0         1.0
b0414              1.0         1.0         1.0         1.0         1.0
b3041              1.0         1.0         1.0         1.0         1.0
b0025              1.0         1.0         1.0         1.0         1.0
b0844              1.0         1.0         1.0         1.0         1.0
b0446              1.0         1.0         1.0         1.0         1.0
b3812              1.0         1.0         1.0         1.0         1.0
b1662              1.0         1.0         1.0         1.0         1.0
b0421              1.0         1.0         1.0         1.0         1.0
...                ...         ...         ...         ...         ...
b4095              1.0         1.0         1.0         1.0         1.0
b4092              1.0         1.0         1.0         1.0         1.0
b2047              1.0         1.0         1.0         1.0         1.0
b4106              1.0         1.0         1.0         1.0         1.0
b4104              1.0         1.0         1.0         1.0         1.0
b4105              1.0         1.0         1.0         1.0         1.0
b0662              1.0         1.0         1.0         1.0         1.0
b2906              1.0         1.0         1.0         1.0         1.0
b1400              1.0         0.0         0.0         0.0         0.0
b2683              1.0         1.0         1.0         1.0         1.0
b2682              1.0         1.0         1.0         1.0         1.0
b1821              1.0         1.0         1.0         1.0         1.0
b2164              1.0         1.0         1.0         1.0         1.0
b1616              1.0         1.0         1.0         1.0         1.0
b1517              1.0         0.0         0.0         1.0         1.0
b0577              1.0         1.0         1.0         1.0         1.0
b0465              1.0         1.0         1.0         1.0         1.0
b3291              1.0         1.0         1.0         1.0         1.0
b4159              1.0         1.0         1.0         1.0         1.0
b2924              1.0         1.0         1.0         1.0         1.0
b1330              1.0         1.0         1.0         1.0         1.0
b0808              1.0         1.0         1.0         1.0         1.0
b2247              1.0         1.0         1.0         0.0         1.0
b0597              1.0         1.0  

### Delete missing genes from copies of the iML1515 model
For each strain, start with the iML1515 model, identify the missing genes from the matrix, and remove them and their associated reactions from the reference model 

In [5]:
#create strain-specific draft models and save them
for strain in hom_matrix.columns:
    
    #Get the list of Gene IDs from the homology matrix dataframe for the current strain without a homolog
    currentStrain=hom_matrix[strain]
    nonHomologous=currentStrain[currentStrain==0.0]
    nonHomologous=nonHomologous.index.tolist()
    
    #s0001 is an artificial gene used in iML1515 for spontaneous reactions and as such has no homologs,
    #However, it is retained for these spontaneous reactions to function
    nonHomologous.remove('s0001')
    
    #Define a list of Gene objects from the base reconstruction to be deleted from the current strain
    toDelete=[]
    for gene in nonHomologous:
        toDelete.append(model.genes.get_by_id(gene))

    #Establish a model copy and use the COBRApy function to remove the appropriate content and save this model
    modelCopy=model.copy()
    remove_genes(modelCopy, toDelete, remove_reactions=True)
    modelCopy.id=str(strain)
    cobra.io.json.save_json_model(modelCopy, str('Models/'+strain+'.json'), pretty=False)

## 2. Update Model Gene Product Rules

In [7]:
#load the geneID matrix from the notebook1 
models=glob('%s/*.json'%'Models')
geneIDs_matrix=pd.read_csv('geneIDs_matrix.csv')
geneIDs_matrix=geneIDs_matrix.set_index('Unnamed: 0')
geneIDs_matrix

CP000946.1 CU651637.1   CP002167.1  CU928163.2    CU928164.2
Unnamed: 0                                                                  
b2551           EcolC_1126  LF82_0891  UM146_03965  ECUMN_2871  ECIAI39_2754
b0870           EcolC_2726  LF82_1236  UM146_13300  ECUMN_1065  ECIAI39_0851
b3368           EcolC_0344  LF82_0414  UM146_16925  ECUMN_3831  ECIAI39_3852
b2436           EcolC_1243  LF82_0980  UM146_04450  ECUMN_2757  ECIAI39_2581
b3500           EcolC_0216  LF82_0914  UM146_17635  ECUMN_3985  ECIAI39_3990
b0945           EcolC_2651  LF82_1793  UM146_12850  ECUMN_1134  ECIAI39_2202
b4467           EcolC_0723  LF82_0833  UM146_01510  ECUMN_3453  ECIAI39_3465
b4468           EcolC_0722  LF82_0832  UM146_01505  ECUMN_3454  ECIAI39_3466
b2979           EcolC_0721  LF82_0831  UM146_01500  ECUMN_3455  ECIAI39_3467
b3916           EcolC_4102  LF82_1619  UM146_19810  ECUMN_4444  ECIAI39_3080
b1260           EcolC_2367  LF82_2309  UM146_10480  ECUMN_1559  ECIAI39_1597
b1261           EcolC_2366  LF82_2310  UM146_10475  ECUMN_1560  ECIAI39_1598
b3426           EcolC_0288  LF82_0866  UM146_17185  ECUMN_3884  ECIAI39_3906
b2242           EcolC_1409  LF82_0864  UM146_05600  ECUMN_2582  ECIAI39_2385
b2243           EcolC_1408  LF82_0865  UM146_05595  ECUMN_2583  ECIAI39_2386
b2241           EcolC_1410  LF82_0863  UM146_05605  ECUMN_2581  ECIAI39_2384
b4054           EcolC_3974  LF82_2339  UM146_20410  ECUMN_4588  ECIAI39_4474
b3770           EcolC_4232  LF82_1105  UM146_18995  ECUMN_4296  ECIAI39_3016
b3281           EcolC_0432  LF82_0144  UM146_16315  ECUMN_3755  ECIAI39_3775
b1692           EcolC_1939  LF82_2870  UM146_08690  ECUMN_1981  ECIAI39_1366
b0243           EcolC_3338  LF82_1726  UM146_16000  ECUMN_0309  ECIAI39_0408
b1613           EcolC_2017  LF82_1270  UM146_09090  ECUMN_1902  ECIAI39_1445
b0414           EcolC_3219  LF82_1880  UM146_15290  ECUMN_0452  ECIAI39_0262
b3041           EcolC_0656  LF82_1878  UM146_01065  ECUMN_3529  ECIAI39_3538
b0025           EcolC_3630  LF82_1882  UM146_22895  ECUMN_0025  ECIAI39_0026
b0844           EcolC_2798  LF82_2658  UM146_13430  ECUMN_1034  ECIAI39_0823
b0446           EcolC_3169  LF82_0340  UM146_15125  ECUMN_0486  ECIAI39_0227
b3812           EcolC_4195  LF82_3390  UM146_19195  ECUMN_4337  ECIAI39_2975
b1662           EcolC_1967  LF82_1879  UM146_08835  ECUMN_1952  ECIAI39_1394
b0421           EcolC_3212  LF82_1125  UM146_15255  ECUMN_0460  ECIAI39_0255
...                    ...        ...          ...         ...           ...
b4095           EcolC_3931  LF82_1652  UM146_20710  ECUMN_4627  ECIAI39_4519
b4092           EcolC_3934  LF82_1655  UM146_20695  ECUMN_4624  ECIAI39_4516
b2047           EcolC_1594  LF82_2427  UM146_06560  ECUMN_2383  ECIAI39_0968
b4106           EcolC_3921  LF82_1642  UM146_20760  ECUMN_4637  ECIAI39_4529
b4104           EcolC_3923  LF82_1644  UM146_20750  ECUMN_4635  ECIAI39_4527
b4105           EcolC_3922  LF82_1643  UM146_20755  ECUMN_4636  ECIAI39_4528
b0662           EcolC_2983  LF82_2351  UM146_14240  ECUMN_0755  ECIAI39_0629
b2906           EcolC_0803  LF82_2417  UM146_01995  ECUMN_3248  ECIAI39_3322
b1400           EcolC_2255  LF82_0261  UM146_22955  ECUMN_0037  ECIAI39_0036
b2683           EcolC_1024  LF82_3128  UM146_03170  ECUMN_3008  ECIAI39_2872
b2682           EcolC_1025  LF82_3135  UM146_03175  ECUMN_3007  ECIAI39_2871
b1821           EcolC_1811  LF82_2928  UM146_08065  ECUMN_2114  ECIAI39_1231
b2164           EcolC_1484  LF82_3015  UM146_05975  ECUMN_2500  ECIAI39_2304
b1616       b1616_ortholog  LF82_2369  UM146_09075  ECUMN_1905  ECIAI39_1442
b1517           EcolC_2141       None         None  ECUMN_1785  ECIAI39_1780
b0577           EcolC_3069  LF82_2585  UM146_14630  ECUMN_0667  ECIAI39_0552
b0465           EcolC_3151  LF82_1156  UM146_15035  ECUMN_0504  ECIAI39_0206
b3291           EcolC_0423  LF82_1399  UM146_16360  ECUMN_3764  ECIAI39_3785
b4159           EcolC_3851  LF82_3455  UM146_21045  ECUMN_4693  ECIAI39_4624
b2924       

In [8]:
#Utilize the geneIDs matrix to update the GPRs in each of the strain-specific models with the proper gene ID

from cobra.manipulation.modify import rename_genes

for mod in models:
    model=cobra.io.load_json_model(mod)
    for column in geneIDs_matrix.columns:
        if column in mod:
            currentStrain=column
    
    IDMapping=geneIDs_matrix[currentStrain].to_dict()
    IDMappingParsed = {k:v for k,v in IDMapping.items() if v != 'None'}
    
    rename_genes(model,IDMappingParsed)
    cobra.io.json.save_json_model(model,mod, pretty=False)

### Examine the draft strain specific model contents

In [9]:
# gather the general information on the draft models
for strain in hom_matrix.columns:
    model=cobra.io.load_json_model(str('Models/'+strain+'.json'))
    print (model.id,'Number of Model Genes:',len(model.genes),'Number of Model Reactions:',len(model.reactions))

CP000946.1 Number of Model Genes: 1484 Number of Model Reactions: 2664
CU651637.1 Number of Model Genes: 1420 Number of Model Reactions: 2613
CP002167.1 Number of Model Genes: 1425 Number of Model Reactions: 2614
CU928163.2 Number of Model Genes: 1460 Number of Model Reactions: 2650
CU928164.2 Number of Model Genes: 1459 Number of Model Reactions: 2622
